In [1]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd
import random
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression

In [5]:
prediction_df = pd.read_csv('results/TCA/tptl.csv')
prediction_df = prediction_df.drop(['Unnamed: 0'], axis = 1)
prediction_df = prediction_df.sort_values(['trg'])

In [7]:

prediction_df

,src,trg,f,pci_20
204,Once,Android-ReactiveLocation,0.519549,0.704204
44,SmoothProgressBar,Android-ReactiveLocation,0.524957,0.598388
144,aho-corasick,Android-ReactiveLocation,0.451817,0.721967
344,Space-Navigation-View,Android-ReactiveLocation,0.473332,0.599967
284,mqtt-client,Android-ReactiveLocation,0.419660,0.648597
...,...,...,...,...
221,easygcm,swipe-button,0.427827,0.665215
241,reflections,swipe-button,0.359221,0.694398
341,Space-Navigation-View,swipe-button,0.376012,0.596244
61,JustAuth,swipe-button,0.442052,0.651389
